In [1]:
import sys
import os
import fastText

from validation import compute_f1

from keras.models import load_model
from keras.callbacks import ModelCheckpoint, Callback

import models
import utils

/home/gwiedemann/miniconda3/envs/kerasenv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2


In [23]:
trainSentences = utils.get_sentences_germeval('../data/GermEVAL/NER-de-train.tsv')
devSentences = utils.get_sentences_germeval('../data/GermEVAL/NER-de-dev.tsv')
testSentences = utils.get_sentences_germeval('../data/GermEVAL/NER-de-test.tsv')

# trainSentences = get_sentences('../data/CONLL/deu/deu_utf.train')
# devSentences = get_sentences('../data/CONLL/deu/deu_utf.testa')
# testSentences = get_sentences('../data/CONLL/deu/deu_utf.testb')

print(len(trainSentences))
print(len(devSentences))
print(len(testSentences))


24000
2200
5100


In [24]:
print(testSentences[0])

[['1951', 'O'], ['bis', 'O'], ['1953', 'O'], ['wurde', 'O'], ['der', 'O'], ['nördliche', 'O'], ['Teil', 'O'], ['als', 'O'], ['Jugendburg', 'O'], ['des', 'O'], ['Kolpingwerkes', 'B-OTH'], ['gebaut', 'O'], ['.', 'O']]


In [25]:
labelSet = set()
characters= set()
models.max_sequence_length = 0

for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for word, label in sentence:
            for char in word:
                characters.add(char)
            labelSet.add(label)
        if len(sentence) > models.max_sequence_length:
            models.max_sequence_length = len(sentence)

In [26]:
print(len(labelSet))
print(models.max_sequence_length)

25
56


In [27]:
# :: Create a mapping for the labels ::
models.label2Idx = {"PADDING_TOKEN":0}
for label in labelSet:
    models.label2Idx[label] = len(models.label2Idx)

In [28]:
print(models.label2Idx)

{'B-OTH': 2, 'I-PER': 5, 'I-ORG': 24, 'I-OTH': 6, 'B-PERderiv': 22, 'B-OTHpart': 17, 'I-ORGpart': 7, 'B-PER': 8, 'I-LOC': 16, 'B-OTHderiv': 14, 'B-LOC': 19, 'I-OTHderiv': 25, 'I-OTHpart': 20, 'B-LOCderiv': 9, 'B-ORGpart': 21, 'B-PERpart': 10, 'I-ORGderiv': 11, 'O': 1, 'I-PERderiv': 12, 'B-ORG': 15, 'B-ORGderiv': 23, 'I-LOCpart': 4, 'I-PERpart': 13, 'B-LOCpart': 18, 'PADDING_TOKEN': 0, 'I-LOCderiv': 3}


In [29]:
# :: Hard coded case lookup ::
models.case2Idx = {'PADDING_TOKEN':0, 'numeric': 1, 'allLower':2, 'allUpper':3, 'initialUpper':4, 'other':5, 'mainly_numeric':6, 'contains_digit': 7}

In [30]:
print(models.case2Idx)

{'allLower': 2, 'initialUpper': 4, 'allUpper': 3, 'mainly_numeric': 6, 'numeric': 1, 'PADDING_TOKEN': 0, 'contains_digit': 7, 'other': 5}


In [31]:
print(type(trainSentences))

<class 'list'>


In [32]:
print(trainSentences[0])

[['Schartau', 'B-PER'], ['sagte', 'O'], ['dem', 'O'], ['"', 'O'], ['Tagesspiegel', 'B-ORG'], ['"', 'O'], ['vom', 'O'], ['Freitag', 'O'], [',', 'O'], ['Fischer', 'B-PER'], ['sei', 'O'], ['"', 'O'], ['in', 'O'], ['einer', 'O'], ['Weise', 'O'], ['aufgetreten', 'O'], [',', 'O'], ['die', 'O'], ['alles', 'O'], ['andere', 'O'], ['als', 'O'], ['überzeugend', 'O'], ['war', 'O'], ['"', 'O'], ['.', 'O']]


In [33]:
models.char2Idx={"PADDING_TOKEN":0, "<S>":1, "</S>":2, "<W>":3, "</W>":4}
for char in characters:
    models.char2Idx[char] = len(models.char2Idx)
models.char2Idx['UNKNOWN'] = len(models.char2Idx)
print(models.char2Idx)

{'`': 5, 'D': 6, 'ħ': 7, '殿': 8, '”': 9, 'E': 10, '&': 11, '算': 12, '±': 184, '©': 14, 'f': 15, ')': 17, 'N': 18, 'ß': 20, 'q': 21, 'ż': 27, 'τ': 25, 'M': 24, 'ı': 284, 'G': 30, '士': 31, '@': 32, 'ê': 34, 'ῦ': 39, 'ь': 37, '별': 188, 'õ': 40, '\u200e': 41, '동': 310, 'ú': 42, '?': 43, '"': 45, 'û': 47, 'ε': 49, 'М': 50, 'ς': 190, '.': 51, '佐': 296, 'ν': 53, 'Þ': 67, 'j': 54, 'π': 19, 'ă': 292, 'ế': 57, 'č': 56, 'ο': 58, '術': 59, '造': 60, 'є': 68, '…': 62, 'ά': 293, '<W>': 3, 'к': 311, 'â': 63, '»': 64, 'ó': 66, '0': 69, 'أ': 13, '«': 209, 'ā': 71, 'V': 72, '/': 73, 'ğ': 75, 'и': 332, '4': 76, 'ɨ': 78, '⊃': 87, 'а': 79, '‘': 80, 'л': 83, 'п': 84, ']': 129, 'm': 85, 'd': 88, 'Ł': 89, 'ا': 90, 'в': 91, '≤': 245, '!': 316, 'T': 16, 'í': 93, 'б': 22, ';': 94, 'F': 95, 'ι': 96, '_': 70, 'B': 97, 'n': 98, '—': 99, 'ю': 100, '→': 101, 'Ž': 131, 'J': 103, '<': 105, 'H': 106, 'β': 107, '¸': 108, 'ś': 26, 'ź': 110, '$': 111, 'İ': 112, 'ḳ': 331, 'æ': 28, '(': 113, '–': 114, 'C': 115, 'ب': 116, 'UNKN

In [34]:
print(trainSentences[0]) 

[['Schartau', 'B-PER'], ['sagte', 'O'], ['dem', 'O'], ['"', 'O'], ['Tagesspiegel', 'B-ORG'], ['"', 'O'], ['vom', 'O'], ['Freitag', 'O'], [',', 'O'], ['Fischer', 'B-PER'], ['sei', 'O'], ['"', 'O'], ['in', 'O'], ['einer', 'O'], ['Weise', 'O'], ['aufgetreten', 'O'], [',', 'O'], ['die', 'O'], ['alles', 'O'], ['andere', 'O'], ['als', 'O'], ['überzeugend', 'O'], ['war', 'O'], ['"', 'O'], ['.', 'O']]


In [35]:
models.ft = fastText.load_model("../embeddings/wiki.de.bin")

In [36]:
print(models.nb_embedding_dims)
print(len(trainSentences[0]))

300
25


In [37]:
models.idx2Label = {v: k for k, v in models.label2Idx.items()}
print(len(models.label2Idx))
print(len(models.idx2Label))

26
26


# Test Model

In [22]:
# import importlib
# importlib.reload(models)

<module 'models' from '/home/gwiedemann/microNER/scripts/models.py'>

In [38]:
tmp_model_filename = 'tmp_3cnn_bi-lstm_mp.h5'
# checkpoint = ModelCheckpoint(tmp_model_filename, verbose=1, save_best_only = True, monitor = 'val_acc')
history = utils.F1History(tmp_model_filename, devSet = devSentences)
model = models.get_model_3cnn()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 32) 10688       char_input[0][0]                 
__________________________________________________________________________________________________
time_distributed_10 (TimeDistri (None, None, 52, 32) 3104        char_embedding[0][0]             
__________________________________________________________________________________________________
time_distributed_13 (TimeDistri (None, None, 52, 32) 4128        char_embedding[0][0]             
__________________________________________________________________________________________________
time_distr

In [20]:
# import importlib
# importlib.reload(utils)
#cprint(models.max_sequence_length)

In [ ]:
model.fit_generator(
    utils.NerSequence(trainSentences, shuffle_data=True, batch_size=32), 
    validation_data = utils.NerSequence(devSentences, batch_size=256), 
    epochs = 10, callbacks = [history]
)

Epoch 1/10
749/750 [============================>.] - ETA: 0s - loss: 0.0783 - acc: 0.9791
New maximum F1 score: 0.7373529960798955 (before: 0) Saving to tmp_3cnn_bi-lstm_mp.h5
750/750 [==============================] - 277s 369ms/step - loss: 0.0782 - acc: 0.9791 - val_loss: 0.0262 - val_acc: 0.9881
Epoch 2/10
749/750 [============================>.] - ETA: 0s - loss: 0.0157 - acc: 0.9881
New maximum F1 score: 0.7723561799886082 (before: 0.7373529960798955) Saving to tmp_3cnn_bi-lstm_mp.h5
750/750 [==============================] - 244s 325ms/step - loss: 0.0157 - acc: 0.9881 - val_loss: 0.0022 - val_acc: 0.9890
Epoch 3/10
156/750 [=====>........................] - ETA: 2:59 - loss: -5.8808e-04 - acc: 0.9892

In [ ]:
print(history.acc)
print(history.f1_scores)

In [ ]:
model.load_weights(tmp_model_filename)

In [ ]:
model.fit_generator(
    utils.NerSequence(trainSentences, shuffle_data=True, batch_size=2048), 
    validation_data = utils.NerSequence(devSentences, batch_size=256), 
    epochs = 5, callbacks = [history]
)

In [ ]:
true_labels, pred_labels = utils.predict_sequences(model, testSentences)
print(compute_f1(pred_labels, true_labels, models.idx2Label))

# Experiments

In [ ]:
f = open('results_3cnn.txt', 'a')
for run_i in range(1,10):
    print("Run " + str(run_i))
    
    tmp_model_filename = 'tmp_generator_NER_3cnn_best.' + str(run_i) + '.h5'

    history = utils.F1History(tmp_model_filename, devSet=devSentences)

    model = models.get_model_3cnn()
    model.fit_generator(
        utils.NerSequence(trainSentences, shuffle_data=True, batch_size=32), 
        validation_data = utils.NerSequence(devSentences, batch_size=256), 
        epochs = 10, callbacks = [history]
    )
    
    model.load_weights(tmp_model_filename)
    
    model.fit_generator(
        utils.NerSequence(trainSentences, shuffle_data=True, batch_size=2048), 
        validation_data = utils.NerSequence(devSentences, batch_size=256), 
        epochs = 10, callbacks = [history]
    )
    
    model.load_weights(tmp_model_filename)
    
    true_labels, pred_labels = utils.predict_sequences(model, testSentences)
    
    pre, rec, f1 = compute_f1(pred_labels, true_labels, models.idx2Label)
    f.write(str(run_i) + "\t" + str(pre) + "\t" + str(rec) +  "\t" + str(f1))
    f.write("\n")
    f.flush()
f.close()

# Save final model

In [ ]:
import shutil, json
# copy file for best run
shutil.copyfile('tmp_3cnn_bi-lstm.h5', '../models/final_model_germeval_outer.h5')
with open("../models/final_model_germeval.indexes", "w") as f:
    json.dump([models.idx2Label, models.label2Idx, models.char2Idx, models.case2Idx], f)

In [ ]:
from keras_contrib.layers import CRF
def create_custom_objects():
    instanceHolder = {"instance": None}
    class ClassWrapper(CRF):
        def __init__(self, *args, **kwargs):
            instanceHolder["instance"] = self
            super(ClassWrapper, self).__init__(*args, **kwargs)
    def loss(*args):
        method = getattr(instanceHolder["instance"], "loss_function")
        return method(*args)
    def accuracy(*args):
        method = getattr(instanceHolder["instance"], "accuracy")
        return method(*args)
    return {"ClassWrapper": ClassWrapper ,"CRF": ClassWrapper, "loss": loss, "accuracy":accuracy}

finalmodel = load_model('../models/final_model_germeval_outer.h5', custom_objects=create_custom_objects())
true_labels, pred_labels = utils.predict_sequences(finalmodel, testSentences)
print(compute_f1(pred_labels, true_labels, models.idx2Label))

In [ ]:
f = open('germeval_output.tsv', 'w', encoding='UTF-8')
for i_sent, sent in enumerate(testSentences):
    for i_tok, tok in enumerate(sent):
        if tok[0] == 'PADDING_TOKEN':
            break
        correctlabel = models.idx2Label[true_labels[i_sent][i_tok]]
        guessedlabel = models.idx2Label[pred_labels[i_sent][i_tok]]
        line = "\t".join([str(i_tok+1), tok[0], correctlabel, correctlabel, guessedlabel, guessedlabel])
        f.write(line + '\n')
    f.write('\n')
f.close